In [ ]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split

In [ ]:
#Base de Treino
treino = pd.read_csv('train.csv')
display(treino)

In [ ]:
#Gerar um relatório
#profile = ProfileReport(treino, title="titanic_treino")
#profile.to_file("titanic_treino.html")

In [ ]:
treino.info()
treino.dtypes.value_counts()

In [ ]:
treino.isnull().sum().sort_values(ascending=False).head(5)

--> Tratar Cabin, Age e Embarked na base de dados de treino

In [ ]:
#Base de Teste
teste = pd.read_csv('test.csv')
teste.head(5)

In [ ]:
teste.info()
teste.dtypes.value_counts()

In [ ]:
teste.isnull().sum().sort_values(ascending=False).head(5)

--> Tratar Cabin, Age e Fare na base de teste

Temos colunas que possuem valores vazios na base de teste que não estã vazias na base de treino (nesse caso, precisariamos tratar essas colunas apenas base de teste)

--> Tratando Cabin, Age e Embarked na base de treino

In [ ]:
#Tratar valores vazios
#Retirar colunas com alta cardinalidade, ou seja, muita diferença
treino.shape

In [ ]:
treino.head(4)

In [ ]:
treino.nunique().sort_values(ascending=False) #é unico
#Colunas com alta cardinalidade e com texto podem tornar a base de dados menos generalizável.

In [ ]:
#Faz tudo na base de treino e na base de teste
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [ ]:
#Coluna de Age, para não excluir os valores vazios pode ser utilizada a média
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()

In [ ]:
#2 valores vazios na coluna de embarque na base de treino
treino.Embarked.value_counts()

In [ ]:
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

In [ ]:
#1 valor vazio na Fare na base de teste
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

In [ ]:
#Verificando valores vazios novamente
treino.isnull().sum().sort_values(ascending=False).head(5)


In [ ]:
#Verificando valores vazios novamente
teste.isnull().sum().sort_values(ascending=False).head(5)


In [ ]:
col_treino_nr = treino.columns[treino.dtypes != 'object']
treino_nr = treino.loc[:, col_treino_nr] #selecionando todas as linhas mas só das colunas do col_treino_nr

In [ ]:
col_teste_nr = teste.columns[teste.dtypes != 'object']
teste_nr = teste.loc[:, col_teste_nr] #selecionando todas as linhas mas só das colunas do col_treino_nr

In [ ]:
treino.info()
display(treino_nr)

Podemos agora criar modelos para classificar esses dados 
-->Árvore de classificação
-->KNN
-->Regressão Logística

In [ ]:
#Separar em treino e validação: 70/30 - Usando sklearn
X = treino_nr.drop(['PassengerId', 'Survived'], axis=1) #retira a que eu quero prever
y = treino_nr.Survived #quem eu quero prever


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42) #chamar de validação pois o teste é o final o que vou prever

In [ ]:
#Árvore de classificação
from sklearn import tree
c_ac = tree.DecisionTreeClassifier(random_state=42)
c_ac = c_ac.fit(X_train,y_train)
y_pred_ac = c_ac.predict(X_val)


In [ ]:
#KNN - Vizinho mais próximo
from sklearn.neighbors import KNeighborsClassifier
c_knn = KNeighborsClassifier(n_neighbors=3)
c_knn = c_knn.fit(X_train, y_train)
y_pred_knn = c_knn.predict(X_val)

In [ ]:
#Regressão Logística
from sklearn.linear_model import LogisticRegression
c_rl = LogisticRegression(random_state=42)
c_rl = c_rl.fit(X_train, y_train)
y_pred_rl = c_rl.predict(X_val)

In [ ]:
#Avaliando os modelos
#Acuracia
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_val, y_pred_ac)

In [ ]:
accuracy_score(y_val, y_pred_knn)

In [ ]:
accuracy_score(y_val, y_pred_rl)

In [ ]:
#Matriz de confusão
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_val, y_pred_ac)

In [ ]:
confusion_matrix(y_val, y_pred_knn)

In [ ]:
confusion_matrix(y_val, y_pred_rl)

Previsão para os dados de teste:

In [ ]:
#Analisar as colunas
X_train.head(3)

In [ ]:
teste_nr.head(3)

In [ ]:
X_teste = teste_nr.drop(['PassengerId'], axis=1)

In [ ]:
y_pred = c_rl.predict(X_teste)

In [ ]:
teste['Survived'] = y_pred

In [ ]:
display(teste)

In [ ]:
base_envio = teste[['PassengerId', 'Survived']]
base_envio.to_csv('Resultados01.csv', index=False)